<a href="https://colab.research.google.com/github/AdeshGurjar02/AdeshGurjar02/blob/main/Copy_of_Video_SD_To_HD_Working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install diffusers scipy ftfy opencv-python torch numpy transformers opencv-python-headless torchvision

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
# rm -rf /root/.cache/huggingface/hub

In [ ]:
# pip install --upgrade transformers huggingface_hub

In [ ]:
import cv2
import numpy as np
from PIL import Image
import torch
from diffusers import StableDiffusionInpaintPipeline

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
def get_video_path():
    return input("Enter the path to the video file: ")

In [ ]:
def read_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames

In [ ]:
video_path = get_video_path()
if video_path:
    frames = read_video(video_path)
    if frames:
        print(f"Total frames read: {len(frames)}")
        print(f"First frame shape: {frames[0].shape}")
    else:
        print("No frames were read from the video. Please check the video path and ensure the video is readable.")

        exit()

Enter the path to the video file: /content/adesh-1.mp3
Total frames read: 1
First frame shape: (408, 338, 3)


In [ ]:
def preprocess_frame(frame, target_size=(1280, 720)):
    h, w, _ = frame.shape
    scale_factor = min(target_size[0] / w, target_size[1] / h)
    new_w, new_h = int(w * scale_factor), int(h * scale_factor)
    resized_frame = cv2.resize(frame, (new_w, new_h))

    top_pad = (target_size[1] - new_h) // 2
    bottom_pad = target_size[1] - new_h - top_pad
    left_pad = (target_size[0] - new_w) // 2
    right_pad = target_size[0] - new_w - left_pad

    padded_frame = cv2.copyMakeBorder(resized_frame, top_pad, bottom_pad, left_pad, right_pad, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    return padded_frame

In [ ]:
preprocessed_frames = [preprocess_frame(frame) for frame in frames]

In [ ]:
def process_frame(model, device, frame):
      if isinstance(frame, Image.Image):
          image = frame
      elif isinstance(frame, np.ndarray):
          if frame.dtype == object:
              frame = frame.astype(np.uint8)
          image = Image.fromarray(frame)
      else:
          raise ValueError(f"Unsupported frame type: {type(frame)}")
      mask = Image.new("L", image.size, 255)  # Full white mask for inpainting entire image

      # Debugging statements to check image and mask
      print(f"Image size: {image.size}, Image mode: {image.mode}")
      print(f"Mask size: {mask.size}, Mask mode: {mask.mode}")

      # Prepare inputs
      image = image.convert("RGB")
      prompt = "Fill in the blank areas in the image with relevant content while preserving the context."

      with torch.no_grad():
          result = model(prompt, image, mask).images[0]

      # Convert result back to BGR format for OpenCV
      result = cv2.cvtColor(np.array(result), cv2.COLOR_RGB2BGR)

      return result



In [ ]:
def write_video(frames, output_path, fps=30):
    if not frames:
        raise ValueError("No frames to write.")

    height, width, layers = frames[0].shape
    size = (width, height)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 format
    out = cv2.VideoWriter(output_path, fourcc, fps, size)

    for frame in frames:
        out.write(frame)

    out.release()
    print(f"Video saved to {output_path}")

In [ ]:
def get_output_video_path():
    return input("Enter the path to the output video file: ")

In [ ]:
import os
import shutil

def clear_hf_cache():
    cache_path = os.path.expanduser("~/.cache/huggingface")
    if os.path.exists(cache_path):
        shutil.rmtree(cache_path)
        print("Hugging Face cache cleared.")
    else:
        print("Hugging Face cache not found.")

clear_hf_cache()

Hugging Face cache cleared.


In [ ]:
from huggingface_hub import hf_hub_download
import os

def download_model_files():
    model_name = "runwayml/stable-diffusion-inpainting"
    files_to_download = [
        "vae/diffusion_pytorch_model.bin",
        "unet/diffusion_pytorch_model.bin",
        "text_encoder/pytorch_model.bin",
        "safety_checker/pytorch_model.bin"
    ]

    for file in files_to_download:
        try:
            local_path = hf_hub_download(repo_id=model_name, filename=file, repo_type="model")
            print(f"Successfully downloaded {file} to {local_path}")
        except Exception as e:
            print(f"Error downloading {file}: {e}")

download_model_files()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

Successfully downloaded vae/diffusion_pytorch_model.bin to /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/51388a731f57604945fddd703ecb5c50e8e7b49d/vae/diffusion_pytorch_model.bin


diffusion_pytorch_model.bin:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Successfully downloaded unet/diffusion_pytorch_model.bin to /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/51388a731f57604945fddd703ecb5c50e8e7b49d/unet/diffusion_pytorch_model.bin


pytorch_model.bin:   0%|          | 0.00/492M [00:00<?, ?B/s]

Successfully downloaded text_encoder/pytorch_model.bin to /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/51388a731f57604945fddd703ecb5c50e8e7b49d/text_encoder/pytorch_model.bin


pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Successfully downloaded safety_checker/pytorch_model.bin to /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/51388a731f57604945fddd703ecb5c50e8e7b49d/safety_checker/pytorch_model.bin


In [ ]:
import torch
from diffusers import StableDiffusionInpaintPipeline

def load_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    try:
        model = StableDiffusionInpaintPipeline.from_pretrained(
            "runwayml/stable-diffusion-inpainting",
            torch_dtype=torch.float16 if device.type == "cuda" else torch.float32,
            use_safetensors=False,  # Explicitly set to False
            safety_checker=None
        ).to(device)
        return model, device
    except Exception as e:
        print(f"Error loading model: {e}")
        return None, device

In [ ]:
# Print frame information for debugging
for i, frame in enumerate(preprocessed_frames):
    print(f"Frame {i}: Type = {type(frame)}, Shape = {frame.shape if hasattr(frame, 'shape') else 'N/A'}, dtype = {frame.dtype if hasattr(frame, 'dtype') else 'N/A'}")

Frame 0: Type = <class 'numpy.ndarray'>, Shape = (720, 1280, 3), dtype = uint8


In [ ]:
# Ensure all frames are numpy arrays with uint8 dtype
preprocessed_frames = [np.array(frame, dtype=np.uint8) if not isinstance(frame, np.ndarray) or frame.dtype != np.uint8 else frame for frame in preprocessed_frames]

In [ ]:
model, device = load_model()
if model is not None:
    print(f"Model loaded successfully on {device}")
else:
    print("Failed to load the model")

model_index.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

text_encoder/config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/748 [00:00<?, ?B/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/51388a731f57604945fddd703ecb5c50e8e7b49d/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/51388a731f57604945fddd703ecb5c50e8e7b49d/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/51388a731f57604945fddd703ecb5c50e8e7b49d/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/51388a731f57604945fddd703ecb5c50e8e7b49d/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
You have disabled the safety checker for <class 'diffusers.pipelin

Model loaded successfully on cpu


In [ ]:
processed_frames = [process_frame(model, device, frame) for frame in preprocessed_frames]

Image size: (1280, 720), Image mode: RGB
Mask size: (1280, 720), Mask mode: L


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Use a local file path for saving the output video
output_video_path = get_output_video_path()
write_video(processed_frames, output_video_path)

Enter the path to the output video file: /content/drive/MyDrive
Video saved to /content/drive/MyDrive
